In [1]:
import math
import pandas_datareader as web

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


In [8]:
from matplotlib import pyplot

In [22]:
A=["AXISBANK.NS","ITC.NS","AAPL","RCOM.NS","AMZN","TTM","YESBANK.NS","TCS","HDB","INFY.NS"]
R=[]
for name in (A):    
    df = web.DataReader(name, data_source='yahoo', start='2012-01-01', end='2020-2-23')
    data = df.filter(['Close'])
    dataset = data.values
    training_data_len = math.ceil( len(dataset) * .8 )
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)
    train_data = scaled_data[0:training_data_len , :]
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 1)))
    model.add(LSTM(50, return_sequences= False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=1, epochs=1)
    test_data = scaled_data[training_data_len - 60: , :]
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
    apple_quote = web.DataReader(name, data_source='yahoo', start='2012-01-01', end='2020-2-20')
    new_df = apple_quote.filter(['Close'])
    last_60_days = new_df[-60:].values
    last_60_days_scaled = scaler.transform(last_60_days)
    X_test = []
    X_test.append(last_60_days_scaled)
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    pred_price = model.predict(X_test)
    pred_price = scaler.inverse_transform(pred_price)
    print(pred_price)
    R.append(r2_score(predictions , y_test))
print(R)

Train on 1542 samples
1542/1542 [==============================] - 60s 39ms/sample - loss: 0.0021
[[703.48285]]
Train on 1542 samples
1542/1542 [==============================] - 72s 47ms/sample - loss: 0.0019
[[211.0064]]
Train on 1578 samples
1578/1578 [==============================] - 66s 42ms/sample - loss: 6.6725e-04
[[315.25095]]
Train on 1542 samples
1542/1542 [==============================] - 66s 43ms/sample - loss: 0.0031
[[-1.0490882]]
Train on 1578 samples
1578/1578 [==============================] - 71s 45ms/sample - loss: 0.0011
[[2061.8635]]
Train on 1578 samples
1578/1578 [==============================] - 68s 43ms/sample - loss: 0.0026
[[11.2823515]]
Train on 1542 samples
1542/1542 [==============================] - 64s 42ms/sample - loss: 0.0016
[[39.594837]]
Train on 1209 samples
1209/1209 [==============================] - 52s 43ms/sample - loss: 0.0019
[[4.8358393]]
Train on 1578 samples
1578/1578 [==============================] - 68s 43ms/sample - loss: 0.0015
[

In [19]:
print("R2-score: %.2f" % r2_score(y_test , y_test) )

R2-score: 1.00


In [23]:
for i in range(10):
    print(A[i],end =" ")
    print(R[i])

AXISBANK.NS 0.6411966458645567
ITC.NS 0.9433065379522931
AAPL 0.9710692804858797
RCOM.NS 0.9050562810122794
AMZN 0.5976359842825302
TTM 0.9295010948573251
YESBANK.NS 0.968600555974628
TCS 0.7524943644029721
HDB 0.8845988871090709
INFY.NS 0.44279786726674686
